In [1]:
import sys
from pathlib import Path

In [4]:
src_path = Path('../src').resolve()

sys.path.append(str(src_path))

In [24]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel
from timescaledb.hyperfunctions import time_bucket
from sqlalchemy import func
from pprint import pprint

In [ ]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.id.desc()).limit(10)
    compiled_query = query.compile(compile_kwargs={'literal_binds': True})
    print(compiled_query)
    

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.id DESC
 LIMIT 10


In [ ]:
with Session(engine) as session:
    bucket = time_bucket('1 minute', EventModel.time)
    pages = ['/about']
    query = (
        select(bucket,
               EventModel.page,
               func.count().label('event_count')
        )
        .where(
            EventModel.page.in_(pages)
        )
        .group_by(bucket, EventModel.page)
        .order_by(
            bucket,
            EventModel.page
        )
    )
    compiled_query = query.compile(compile_kwargs={'literal_binds': True})
    results = session.exec(query).fetchall()
    pprint(results)

[(datetime.datetime(2025, 4, 7, 21, 21, tzinfo=datetime.timezone.utc), '/about', 1),
 (datetime.datetime(2025, 4, 7, 21, 22, tzinfo=datetime.timezone.utc), '/about', 1015),
 (datetime.datetime(2025, 4, 7, 21, 23, tzinfo=datetime.timezone.utc), '/about', 959)]
